# Where to host your Trading Bot?

#### Architecture
The general idea is in using some host as an environment which will be work by itself without necessity to involve the human in the process. It means that Trading Bot works while you lay on the beach.

To let it happen, let’s identify major actors i.e. the scripts.

- Screener — responsible for scanning stocks on the stock market to find a pattern which is suitable for trading. We scan the first 500 stocks from NASDAQ index. Each stock is being testing by two technical analysis indicators — BBANDS and RSI;
- Oracular — responsible for stock prices prediction for the upcoming 3 days. In this prediction we use LSTM neural network and Tensorflow model. To learn more about this component, consider reading one of the previous articles in this sequence (How to predict stock market prices with Tensorflow and LSTM neural network);
- Trader — it is the final script out of these three. This one is responsible for submitting orders on the trading platform using Trading API as well as for buying and selling stocks. Everything that this script does is reading preliminary prepared list of stocks and its predictions from Pub/Sub infrastructure and then does the trading by itself.

![Image not found](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*m7vnMN6IBvKh3tozuUAmRw.png)

We have to understand that all three scripts have an order in which they are being launched. The first one is the ‘Screener’ cause it has to find valuable stocks for trading, then follows the ‘Oracular’ to make its predictions for the defined list of preliminary prepared stocks by ‘Screener’. And the final one is ‘Trader’ which prepares and does trades by removing messages (stocks and predictions) from the Pub/Sub’s topics.

Colab’s links for the full parts of code is at the end of the article.

#### Pub/Sub’s infrastructure

Major component of the system is “GCP Pub/Sub topics”. It allows us to establish connectivity between all components. Let’s discuss them here.

Si bien la guía hace el despliegue en GCP, vamos a tratar de hacer el despliegue en AWS.

We have two topics so far:

- Screener topic — is used by ‘Screener’ to keep stocks which were found during the screening process for ‘Oracular’. In our experiment we save just only the first three stocks from the list into Pub/Sub topic;
- Oracular topic — as well as ‘Screener topic’ is used by ‘Trader’ to make its trades. The component reads the predictions from the ‘Oracular topic’ to use them in trading itself. Just only ‘Oracular’ publishes predictions into this topic.

Also we have the appropriate ‘Subscriptions’ which are created automatically during the creation process. We will look on how to create a Topic later in this article. So far, let’s consider the general scheme.


![Image not found](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*CVXeCR7xLlPRsymvv16Ggg.png)

According to the picture above, we have three major blocks. They are: ‘Screener’, ‘Oracular’ and ‘Trader’. Each block has its own scheduler (GCP Job) and Function. Scheduler runs Function with particular schedule. En AWS se lo va a hacer con EventBridge ( https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-create-rule-schedule.html ).

Using the subscriptions, each component reads/writes data from/to particular topics which were mentioned above. ‘Trader’ just reads data from ‘Oracular’ and ‘Screener’ at the same time checking that data ‘as read’. The ‘Oracular’ reads from ‘Screener topic’ for its tasks, while the ‘Screener’ never reads — it is publisher. It just writes new stocks that were found during the screening process.

As listed in the listing of the script ‘Trader’ we have defined two topics.

Se crean, entonces, 2 AWS SNS Topics

##### Recursos creados

- 3 SNS Topics
- 3 SNS Subscriptions - pendiente
- 3 EventBridge Schedules - pendiente
- 3 lambda functions
